In [ ]:
# import os.path as osp

import polars as pl
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.nn import GCNConv
from torch_geometric.seed import seed_everything

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
seed_everything(42)

In [ ]:
df = pl.read_parquet(
    "/home/aarcara/soccerai/soccerai/data/resources/raw/dataset.parquet"
)

index
gameId
gameEventId
possessionEventId
startTime
endTime


In [ ]:
df["duration"].mean()

In [ ]:
df["duration"].var()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df["playerRole"].to_dummies()

In [ ]:
# X["possessionEventType"].value_counts()

In [ ]:
402447 / 22

In [ ]:
# X

In [ ]:
df.with_columns(pl.col("frameTime").str.split(":").list.get(0))

In [ ]:
# X["possessionEventType"].value_counts()

In [ ]:
# labeled_events_df = event_df.with_columns(
#     pl.when(pl.col("index").is_in(pos_indices))
#     .then(1)
#     .when(pl.col("index").is_in(neg_indices))
#     .then(0)
#     .otherwise(None)
#     .alias("label")
# ).filter(pl.col("label").is_not_null())


In [ ]:
# X["playerRole"].value_counts()

In [ ]:
X = df.drop(
    [
        "gameEventType",
        "index",
        "gameId",
        "startTime",
        "endTime",
        "index_right",
        "gameId_right",
        "jerseyNum",
        "visibility",
        "z",
        "videoUrl",
        "homeTeamName",
        "awayTeamName",
        "Age Info",
        "Full Name",
        "Height",
        "birth_date",
    ]
)
X = X.filter(pl.col("team").is_not_null())
X = X.filter(pl.col("possessionEventType") != "SH")
X = X.filter(pl.col("playerName").is_not_null())
X = X.with_columns(
    (
        pl.col("frameTime").str.split(":").list.get(0).cast(pl.UInt16) * 60
        + pl.col("frameTime").str.split(":").list.get(1).cast(pl.UInt16)
    ).alias("frameTime"),
    (
        pl.when(pl.col("playerName") == pl.col("playerName_right")).then(1).otherwise(0)
    ).alias("ballPossession"),
    (pl.col("Weight").str.replace("kg", "").cast(pl.UInt8)),
    (pl.col("height_cm").cast(pl.UInt8)),
)
X = X.drop(["playerName", "playerName_right"])
X = X.to_dummies(["possessionEventType"])
X

teniamo:
per il tempo
- frameTime (convertire in secondi)
- duration
- possessionEventType (OneHot)
- playerName va trasformato in un valore booleano che indica il giocatore che ha la palla ma di quel nodo (ovvero playerName == playerName_right)
- si tiene team per partizionare i giocatori in due squadre (OneHot) (perchè non ordinale? Trello)
- indicare anche il Team che ha la palla (booleano)
- x, y
- velocity
- direction
- playerId (oneHot)
- playerRole (oneHot)
- height
- weight
- age
- goals
- shots


togliamo:
- index
- gameId
- gameEventId
- possessionEventId
- startTime
- endTime
- gameEventType
- teamName va droppato per i motivi spiegati (da scrivere su Trello)
- videoUrl
- z si droppa
- tutte le righe con la palla si droppano
- label si droppa da X
- index_right
- gameId_right
- jerseyNum
- visibility
- homeTeamName
- awayTeamName
- playerName_right
- Full Name
- birthdate



vogliamo far sapere alla rete che squadra sta giocando? e forza una sorta di conoscenza della squadra all'intenro della rete stessa?
SIAMO DUBBIOSI

In [ ]:
df

In [ ]:
from enum import Enum


class SupportedConnectionMode(Enum):
    FULLY_CONNECTED = "fully_connected"


class WorldCup2022Dataset(InMemoryDataset):
    def __init__(self, root, mode, transform=None):
        self.mode = mode
        super().__init__(root, transform)

    @property
    def raw_file_names(self):
        return "dataset.parquet"

    @property
    def processed_file_names(self):
        return "data.pt"

    def process(self):
        df = pl.read_parquet(self.raw_paths[0])
        data_list = self.convert_df_to_data_list(df)

        anomalous_frames = (
            df.group_by("gameEventId", "possessionEventId")
            .len()
            .filter(pl.col("len") > 23)
        )
        event_ids_to_drop = anomalous_frames["gameEventId"].to_list()
        possession_ids_to_drop = anomalous_frames["possessionEventId"].to_list()
        df_cleaned = df.filter(
            ~(
                (pl.col("gameEventId").is_in(event_ids_to_drop))
                & (pl.col("possessionEventId").is_in(possession_ids_to_drop))
            )
        )
        data_list = self.convert_df_to_data_list(df_cleaned)
        self.save(data_list, self.processed_paths[0])

    def create_edge_index(self):
        if self.mode == SupportedConnectionMode.FULLY_CONNECTED:
            src = []
            dst = []
            for i in range(22):
                src += [i] * 21
                for j in range(22):
                    if i != j:
                        dst += [j]
            return torch.tensor([src, dst], dtype=torch.long)

    def convert_df_to_data_list(self, df: pl.DataFrame):
        data_list = []
        for _, event_df in df.group_by(["gameEventId", "possessionEventId"]):
            edge_idx = self.create_edge_index()

            x = []
            for i in range(22):
                x.append(list(event_df.row(i, named=True).values()))
            x = torch.tensor(x, dtype=torch.float)

            y = torch.tensor(event_df["label"][0], dtype=torch.long)

            data_list.append(Data(x=x, edge_index=edge_idx, y=y))
        return data_list


dataset = WorldCup2022Dataset(
    root="/home/aarcara/soccerai/soccerai/data/resources", mode="fully_connected"
)

In [ ]:
df

In [ ]:
anomalous_frames = (
    df.group_by("gameEventId", "possessionEventId")
    .agg(pl.col("index").count())
    .filter(pl.col("index") != 23)
)
event_ids_to_drop = anomalous_frames["gameEventId"].to_list()
possession_ids_to_drop = anomalous_frames["possessionEventId"].to_list()
df_cleaned = df.filter(
    ~(
        (pl.col("gameEventId").is_in(event_ids_to_drop))
        & (pl.col("possessionEventId").is_in(possession_ids_to_drop))
    )
)
df_cleaned.group_by("gameEventId", "possessionEventId").agg(
    pl.col("index").count()
).filter(pl.col("index") != 23)

In [ ]:
class GCN(torch.nn.Module):
    def init(self, dataset):
        super().init()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
df["label"].value_counts()

In [ ]:
df.describe()